# First Simple Model 
Here we'll do a simple model using Librosa for audio feature extraction and Tensorflow for a Convolution Neural Network to predict human emotions with recorded speech data. 
inpsired by : https://www.kaggle.com/ejlok1/audio-emotion-part-3-baseline-model

In [1]:
# import necessary libraries 
import numpy as np #? 
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import librosa 
import librosa.display
# sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Importing required libraries 
# Keras

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras import regularizers
from keras.callbacks import  ModelCheckpoint
from tensorflow.keras.layers import *

from tensorflow.keras import optimizers


In [2]:
# import csv with metadata 
ref  = pd.read_csv('/Users/stephen/Emotion_Dectection/data/RAVDESS/Ravdess.csv',index_col=[0])
ref.head()

,Actor,classID,Class
File_name,,,
/03-01-05-01-02-01-16.wav,_16,5,angry
/03-01-06-01-02-02-16.wav,_16,6,fear
/03-01-06-02-01-02-16.wav,_16,6,fear
/03-01-05-02-01-01-16.wav,_16,5,angry
/03-01-07-01-01-01-16.wav,_16,7,disgust


In [3]:
ref.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1440 entries, /03-01-05-01-02-01-16.wav to /03-01-01-01-02-02-08.wav
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Actor    1440 non-null   object
 1   classID  1440 non-null   int64 
 2   Class    1440 non-null   object
dtypes: int64(1), object(2)
memory usage: 45.0+ KB


In [5]:
df = pd.DataFrame(columns=['feature'])

counter = 0
for index, path in enumerate(ref.index): 
    X, sample_rate = librosa.load(path, sr=16000)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40), axis=0)
    df.loc[counter] = [mfccs]
    counter=counter+1
print(len(df))
df.head()

/Users/stephen/opt/anaconda3/envs/torch_env/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


FileNotFoundError: [Errno 2] No such file or directory: '/03-01-05-01-02-01-16.wav'

In [ ]:
# Now extract the mean bands to its own feature columns
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df.head()

In [ ]:
# replace NA with 0
df=df.fillna(0)
print(df.shape)
df[:5]

In [ ]:
# split the data, changed varriable names to avoid conflicts
X = df.drop(columns=['Emotions', 'Path'])
y = df['Emotions']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle=True, random_state=42)
X_train
#y

In [ ]:
print("X_train size : " + str(X_train.shape))
print("y_train size : " + str(y_train.shape))
print("X_test size : " + str(X_test.shape))
print("y_test size : " + str(y_test.shape))

In [ ]:
# Lts do data normalization 
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

# Check the dataset now 
X_train

In [ ]:
# Lets few preparation steps to get it into the correct format for Keras 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))
y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)
#print(y_train[0:10])
#print(y_test[0:10])

# Pickel the lb object for future use 
filename = 'labels'
outfile = open(filename,'wb')
pickle.dump(lb,outfile)
outfile.close()

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

In [ ]:
# New model
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(8)) # Target class number
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.RMSprop(learning_rate=0.00001, decay=1e-6)
model.summary()

In [ ]:
#adam = optimizers.Adam(learning_rate=1e-4)

model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(X_train, y_train, batch_size=16, epochs=100, validation_data=(X_test, y_test))

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()